In [1]:
import yfinance as yf
import pandas as pd
import backtrader as bt
from datetime import datetime
import os
import sys
import json
# from backtrader.analyzers import Returns,DrawDown,SharpeRatio,TradeAnalyzer


In [2]:
# import strategies
sys.path.append('../src/')
from src.strategies.Test_Strategy import TestStrategy
from src.strategies.SMA import SMA
from src.strategies.SMA_RSI import SMA_RSI
from src.strategies.SMA_CrossOver import SmaCrossOver
from src.backtesting.run_backtest import run_test, prepare_cerebro, prepare_and_run_backtest

In [3]:
# def prepare_cerebro(asset,strategy,data_path,start_date:str,end_date:str=datetime.now(),cash:int=100000,commission:float=0)->bt.Cerebro:
#     if end_date ==None:
#         end_date= datetime.strftime(datetime.now(),"%Y-%m-%d")

#     cerebro = bt.Cerebro()
#     cerebro.broker.setcash(cash)
#     cerebro.broker.setcommission(commission=commission)
#     cerebro.addstrategy(strategy)
    
#     isExist = os.path.exists(data_path)
#     if not isExist:
#         data= yf.download(asset,start_date,end=end_date)
#         data.to_csv(data_path)
    
#     data = bt.feeds.YahooFinanceCSVData(
#         dataname=data_path,
#         fromdate=datetime.strptime(start_date,"%Y-%m-%d"),
#         todate=datetime.strptime(end_date,"%Y-%m-%d"),
#         reverse=False
#     )

#     cerebro.adddata(data)
#     # cerebro.addanalyzer(AnnualReturn)
#     cerebro.addanalyzer(TradeAnalyzer)
#     return cerebro

In [4]:
# def run_test(cerebro:bt.Cerebro):

#     result={}

#     cerebro.addanalyzer(SharpeRatio, _name='sharpe')
#     cerebro.addanalyzer(Returns, _name='returns')
#     cerebro.addanalyzer(DrawDown, _name='draw')
#     cerebro.addanalyzer(TradeAnalyzer, _name='trade')
    
#     starting = cerebro.broker.getvalue()
#     res=cerebro.run()
#     final=cerebro.broker.getvalue()

#     thestrat = res[0]

#     sharpe=thestrat.analyzers.sharpe.get_analysis()
#     return_val=thestrat.analyzers.returns.get_analysis()
#     drawdown=thestrat.analyzers.draw.get_analysis()
#     trade=thestrat.analyzers.trade.get_analysis()

#     result["sharpe_ratio"]=sharpe['sharperatio']
#     result["return"]=return_val['rtot']
#     result['max_drawdown'] = drawdown['max']['drawdown']
    
#     try:
#         result['win_trade']=trade['won']['total']
#     except:
#         result['win_trade']="Undefined"
    
#     try:
#         result['loss_trade']=trade['lost']['total']
#     except:
#         result['loss_trade']="Undefined"

#     try:
#         result['total_trade']=trade['total']['total']
#     except:
#         result['total_trade']="Undefined"

#     result['start_portfolio']=starting
#     result['final_portfolio']=final

#     return result

In [5]:
# cerebro=prepare_cerebro('BTC',TestStrategy,"../data/BTC-USD.csv","2024-04-19","2024-06-19")

{
  "asset": "AAPL",
  "strategy": "SMA_RSI",
  "start_date": "2018-12-19",
  "end_date": "2024-06-19",
  "cash": 100000,
  "commission": 0.005
}


# cerebro=prepare_and_run_backtest('GOOGL',SmaCrossOver,"../data/BTC-USD.csv","2006-12-19","2024-06-19", 100000, 0.005)

result=prepare_and_run_backtest('GOOGL',SmaCrossOver,"../data/BTC-USD.csv","2006-12-19","2024-06-19", 100000, 0.005)

# result=run_test(cerebro)
# results = prepare_and_run_backtest()
result

[*********************100%%**********************]  1 of 1 completed


{'sharpe_ratio': -60.00145766332464,
 'return': 0.001536165095361336,
 'max_drawdown': 0.023527330351966216,
 'win_trade': 6,
 'loss_trade': 4,
 'total_trade': 11,
 'start_portfolio': 100000,
 'final_portfolio': 100153.7345601368}

### Send the backend results to kafka topic